In [28]:
import QuantLib as ql


In [27]:
euriborTermStructure = ql.RelinkableYieldTermStructureHandle()
euribor3m = ql.Euribor3M(euriborTermStructure)

In [30]:
todaysDate = ql.Date(23, 5, 2006)

In [31]:
ql.Settings.instance().evaluationDate = todaysDate

In [32]:
calendar = euribor3m.fixingCalendar()

In [33]:
fixingDays = euribor3m.fixingDays()

In [34]:
settlementDate = calendar.advance(todaysDate, fixingDays, ql.Days)

In [37]:
todaysDate

Date(23,5,2006)

In [38]:
settlementDate

Date(25,5,2006)

In [39]:
threeMonthFraQuote = [None for i in range(10)]

In [40]:
threeMonthFraQuote[1]=0.030
threeMonthFraQuote[2]=0.031
threeMonthFraQuote[3]=0.032
threeMonthFraQuote[6]=0.033
threeMonthFraQuote[9]=0.034

In [41]:
threeMonthFraQuote

[None, 0.03, 0.031, 0.032, None, None, 0.033, None, None, 0.034]

In [42]:
fra1x4Rate = ql.SimpleQuote(threeMonthFraQuote[1])
fra2x5Rate = ql.SimpleQuote(threeMonthFraQuote[2])
fra3x6Rate = ql.SimpleQuote(threeMonthFraQuote[3])
fra6x9Rate = ql.SimpleQuote(threeMonthFraQuote[6])
fra9x12Rate = ql.SimpleQuote(threeMonthFraQuote[9])

In [43]:
h1x4 = ql.RelinkableQuoteHandle()
h2x5 = ql.RelinkableQuoteHandle()
h3x6 = ql.RelinkableQuoteHandle()
h6x9 = ql.RelinkableQuoteHandle()
h9x12 = ql.RelinkableQuoteHandle()

In [45]:
h1x4.linkTo(fra1x4Rate)
h2x5.linkTo(fra2x5Rate)
h3x6.linkTo(fra3x6Rate)
h6x9.linkTo(fra6x9Rate)
h9x12.linkTo(fra9x12Rate)

In [46]:
fraDayCounter = euribor3m.dayCounter()

In [47]:
convention = euribor3m.businessDayConvention()

In [49]:
endOfMonth = euribor3m.endOfMonth()

In [51]:
fra1x4 = ql.FraRateHelper(h1x4, 1, 4,
                         fixingDays, calendar, convention,
                         endOfMonth, fraDayCounter)
fra2x5 = ql.FraRateHelper(h2x5, 2, 5,
                         fixingDays, calendar, convention,
                         endOfMonth, fraDayCounter)
fra3x6 = ql.FraRateHelper(h3x6, 3, 6,
                         fixingDays, calendar, convention,
                         endOfMonth, fraDayCounter)
fra6x9 = ql.FraRateHelper(h6x9, 6, 9,
                         fixingDays, calendar, convention,
                         endOfMonth, fraDayCounter)
fra9x12 = ql.FraRateHelper(h9x12, 9, 12,
                         fixingDays, calendar, convention,
                         endOfMonth, fraDayCounter)

In [53]:
termStructureDayCounter = ql.ActualActual(ql.ActualActual.ISDA)

In [54]:
tolerance = 1.0e-15

In [57]:
fraInstruments = ql.RateHelperVector()
fraInstruments.append(fra1x4)
fraInstruments.append(fra2x5)
fraInstruments.append(fra3x6)
fraInstruments.append(fra6x9)
fraInstruments.append(fra9x12)

In [61]:
fraTermStructure = ql.PiecewiseLogLinearDiscount(settlementDate, fraInstruments,
                                         termStructureDayCounter)

In [62]:
discountingTermStructure = ql.RelinkableYieldTermStructureHandle()

In [63]:
discountingTermStructure.linkTo(fraTermStructure)

In [64]:
fraCalendar = euribor3m.fixingCalendar()

In [66]:
fraBusinessDayConvention = euribor3m.businessDayConvention()

In [68]:
fraFwdType = ql.Position.Long

In [69]:
fraNotional = 100.0

In [70]:
FraTermMonths = 3

In [71]:
monthsToStart = [ 1, 2, 3, 6, 9 ]

In [72]:
euriborTermStructure.linkTo(fraTermStructure)

In [82]:
for i, _ in enumerate(monthsToStart):
    #print(i)
    fraValueDate = fraCalendar.advance(settlementDate,monthsToStart[i],ql.Months,
                                       fraBusinessDayConvention)
    fraMaturityDate = fraCalendar.advance(fraValueDate,FraTermMonths,ql.Months,
                                            fraBusinessDayConvention)
    fraStrikeRate = threeMonthFraQuote[monthsToStart[i]]
    myFRA = ql.ForwardRateAgreement(fraValueDate, fraMaturityDate,
                                       fraFwdType,fraStrikeRate,
                                       fraNotional, euribor3m,
                                       discountingTermStructure)
    print("3m Term FRA, Months to Start: ",monthsToStart[i])

    print("strike FRA rate: ",fraStrikeRate)

    print("FRA 3m forward rate: ",myFRA.forwardRate())

    print("FRA market quote: ",threeMonthFraQuote[monthsToStart[i]])

    print("FRA spot value: ",myFRA.spotValue())
    print("FRA forward value: ",myFRA.forwardValue())

    print("FRA implied Yield: ",myFRA.impliedYield(myFRA.spotValue(),
                               myFRA.forwardValue(),
                               settlementDate,
                               ql.Simple,
                               fraDayCounter))

    print("market Zero Rate: ",discountingTermStructure.zeroRate(fraMaturityDate,
                                               fraDayCounter,
                                               ql.Simple))
    print("FRA NPV [should be zero]: ",myFRA.NPV())
    print()

3m Term FRA, Months to Start:  1
strike FRA rate:  0.03
FRA 3m forward rate:  3.000000 % Actual/360 simple compounding
FRA market quote:  0.03
FRA spot value:  99.73470289713771
FRA forward value:  100.76666666666654
FRA implied Yield:  3.003993 % Actual/360 simple compounding
market Zero Rate:  3.003993 % Actual/360 simple compounding
FRA NPV [should be zero]:  -1.2658787651090532e-13

3m Term FRA, Months to Start:  2
strike FRA rate:  0.031
FRA 3m forward rate:  3.100000 % Actual/360 simple compounding
FRA market quote:  0.031
FRA spot value:  99.49488539115336
FRA forward value:  100.79222222222155
FRA implied Yield:  3.068054 % Actual/360 simple compounding
market Zero Rate:  3.068054 % Actual/360 simple compounding
FRA NPV [should be zero]:  -6.593132338113953e-13

3m Term FRA, Months to Start:  3
strike FRA rate:  0.032
FRA 3m forward rate:  3.200000 % Actual/360 simple compounding
FRA market quote:  0.032
FRA spot value:  99.23916639100246
FRA forward value:  100.83555555555502
